In [1]:
import os
os.chdir(r'C:\Users\Navid\Documents\keyword_extraction_food_delivery')

In [2]:
from keras.preprocessing.text import one_hot
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Flatten
from keras.layers.embeddings import Embedding

Using TensorFlow backend.
E:\Anaconda\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
E:\Anaconda\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
E:\Anaconda\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
E:\Anaconda\lib\site-packages\tensorflow\python\framework\dtypes.py:519: FutureWarning: Passing (type, 1) or '1type' as

In [3]:
import numpy as np
import pandas as pd
df = pd.read_csv('skipthedishes1')
df.head()

,Unnamed: 0,feedback,rating
0,0,App over the years has just been awful. Custom...,Rated 2 stars out of five stars
1,1,I used the app tonight to access a $10 credit ...,Rated 1 stars out of five stars
2,2,"I usually use it for pick up, but customer ser...",Rated 2 stars out of five stars
3,3,Since this app came out I've had nothing but i...,Rated 2 stars out of five stars
4,4,"Does the job, could use a dark mode. Also in m...",Rated 3 stars out of five stars


In [4]:
df.drop('Unnamed: 0', axis=1, inplace=True)
df.dropna(inplace=True)

In [5]:
df['rating'] = df['rating'].str.extract('(\d)').astype('int32')

In [6]:
df.head()

,feedback,rating
0,App over the years has just been awful. Custom...,2
1,I used the app tonight to access a $10 credit ...,1
2,"I usually use it for pick up, but customer ser...",2
3,Since this app came out I've had nothing but i...,2
4,"Does the job, could use a dark mode. Also in m...",3


In [7]:
df = df[df.rating != 3]
df.rating.value_counts()

5    730
1    535
4    280
2    268
Name: rating, dtype: int64

In [8]:
df['positively_rated'] = df['rating'].apply(lambda x: 1 if x == 4 or x == 5 else 0)

In [9]:
df.head()

,feedback,rating,positively_rated
0,App over the years has just been awful. Custom...,2,0
1,I used the app tonight to access a $10 credit ...,1,0
2,"I usually use it for pick up, but customer ser...",2,0
3,Since this app came out I've had nothing but i...,2,0
5,Pretty good selection of restaurants but bette...,2,0


In [19]:
docs = np.array(df.feedback)
mean_length = np.mean([len(x) for x in docs])
median_length = np.median(sorted([len(x) for x in docs]))
print('median length of feedbacks: ', median_length)
print('mean length of feedbacks: ', mean_length)

median length of feedbacks:  275.0
mean length of feedbacks:  283.573083287369


In [20]:
labels = np.array(df.positively_rated)

In [22]:
vocab_size = 500
encoded_docs = [one_hot(d, vocab_size) for d in docs]
# print(encoded_docs)

In [23]:
max_length = 280
padded_docs = pad_sequences(encoded_docs, maxlen= max_length, padding= 'post', truncating= 'post')
print(padded_docs[0])

[240 476 466  53 246 160 405  88  68 311  71 227 195 230 304 148 308  36
 172 441 466 320  71 187 195 141 147 256 172 466 198 466 240 160 476 455
 220 322 452 412 195 315 469 154 148 308  36 172  68 311 468 356 455 484
 390 359  45 466 320 467 414 356 466 443 157 385 171 336 331 195 484 154
  95 293  71 377  81 466 290 220 433 220 119 293 287 373 179 254  23 477
 171 304  20 464 205 429  76 204   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   

In [24]:
model = Sequential([
    Embedding(input_dim= vocab_size, output_dim= 32, input_length= max_length),
    Flatten(),
    Dense(1, activation= 'sigmoid')
])

In [25]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 280, 32)           16000     
_________________________________________________________________
flatten_1 (Flatten)          (None, 8960)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 8961      
Total params: 24,961
Trainable params: 24,961
Non-trainable params: 0
_________________________________________________________________


In [26]:
model.compile(optimizer= 'adam', loss= 'binary_crossentropy', metrics= ['accuracy'])

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [28]:
model.fit(padded_docs, labels, validation_split= 0.1, epochs= 10, verbose=2)


Train on 1631 samples, validate on 182 samples
Epoch 1/10
 - 1s - loss: 0.6212 - accuracy: 0.7149 - val_loss: 0.5212 - val_accuracy: 0.7582
Epoch 2/10
 - 0s - loss: 0.4128 - accuracy: 0.8387 - val_loss: 0.3097 - val_accuracy: 0.9505
Epoch 3/10
 - 0s - loss: 0.2113 - accuracy: 0.9669 - val_loss: 0.1437 - val_accuracy: 0.9835
Epoch 4/10
 - 0s - loss: 0.0953 - accuracy: 0.9896 - val_loss: 0.0684 - val_accuracy: 0.9945
Epoch 5/10
 - 0s - loss: 0.0479 - accuracy: 0.9969 - val_loss: 0.0374 - val_accuracy: 1.0000
Epoch 6/10
 - 0s - loss: 0.0270 - accuracy: 1.0000 - val_loss: 0.0228 - val_accuracy: 1.0000
Epoch 7/10
 - 0s - loss: 0.0172 - accuracy: 1.0000 - val_loss: 0.0155 - val_accuracy: 1.0000
Epoch 8/10
 - 0s - loss: 0.0119 - accuracy: 1.0000 - val_loss: 0.0110 - val_accuracy: 1.0000
Epoch 9/10
 - 0s - loss: 0.0087 - accuracy: 1.0000 - val_loss: 0.0082 - val_accuracy: 1.0000
Epoch 10/10
 - 0s - loss: 0.0066 - accuracy: 1.0000 - val_loss: 0.0065 - val_accuracy: 1.0000


In [31]:
model.get_weights()

[array([[ 0.01297927,  0.05093938, -0.04331497, ..., -0.0039606 ,
         -0.0540129 , -0.04274666],
        [-0.08186789, -0.13716243,  0.16999882, ..., -0.06903639,
          0.10041535,  0.15067331],
        [-0.20413212, -0.16228415,  0.02500713, ..., -0.06051598,
         -0.03373631,  0.1679509 ],
        ...,
        [-0.0650264 , -0.14033146,  0.16863555, ..., -0.19300562,
          0.12779555, -0.11154881],
        [ 0.00030326,  0.08588486, -0.06879272, ...,  0.13245508,
         -0.10163922, -0.02997879],
        [-0.14555812, -0.14446639,  0.04636353, ..., -0.06212042,
         -0.03762136, -0.03040559]], dtype=float32),
 array([[ 0.24249066],
        [-0.15573773],
        [-0.29526523],
        ...,
        [-0.01521237],
        [-0.01874959],
        [-0.00359371]], dtype=float32),
 array([-0.0005948], dtype=float32)]